In [ ]:
# ! sh ../bin/install_requirements_databricks.sh
# dbutils.library.restartPython()

#### Import libraries, modules, and config

In [ ]:
import sys
import os

cwd = os.getcwd()
repo_path = os.path.abspath(os.path.join(cwd, '..'))
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from lib.llm.model import (
    model_api_client,
    make_news_impact_on_portfolio,
    make_reasons_of_impact_on_portfolio,
    make_title_from_news,
    make_summary_from_news,
    make_news_impact_on_commodities,
    make_reasons_of_impact_on_commodities,
)
from lib.vector.querying import retrieve_from_news
import pandas as pd
import tomli
import chromadb as db 
from langchain_chroma import Chroma
from lib.embedding.custom_embedding import CustomHuggingFaceEmbeddings

In [ ]:
with open(os.path.join(os.path.dirname(os.path.abspath("")), "config", "config.toml"), "rb") as f:
    config = tomli.load(f)

#### Connect to DBRX & Load Embdedding model

In [ ]:
llm_client = model_api_client(config['models']['llm_model_api'])
embedding_model = CustomHuggingFaceEmbeddings(model_name=config['models']['embdelling_model_name'])  # sentence-transformers/all-MiniLM-l6-v2

#### Load User Portfolio & Commodities

In [ ]:
if config['general']["UPDATE_USER_PORTFOLIO"]:
    user_portfolio_df = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['stocks']["location"],config["data"]['stocks']["filename"])).sample(15)['ticker'].rename('ticker').reset_index(drop=True)
    user_portfolio_df.to_csv(os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['user']["location"], config["data"]['user']['user_portfolio']["location"],config["data"]['user']['user_portfolio']["filename"]), index=False)
else:
    user_portfolio_df = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['user']["location"],config["data"]['user']['user_portfolio']["location"],config["data"]['user']['user_portfolio']["filename"]))['ticker']

In [ ]:
commodities_df = pd.read_csv(os.path.join(repo_path, config["data"]["location"], config["data"]['user']["location"], config["data"]['user']['commodities']["location"],config["data"]['user']['commodities']["filename"]))

#### Connect to Vector Databases
- stocks
- news

In [ ]:
stocks_collection_name = config["data"]['vector_db']["stocks"]["stocks_collection_name"]
stocks_db_path = os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['vector_db']["location"],config["data"]['vector_db']["stocks"]["location"])
stocks_chroma_client = db.PersistentClient(path=stocks_db_path)
stocks_langchain_chroma = Chroma(
    client=stocks_chroma_client,
    collection_name= stocks_collection_name,
    embedding_function=embedding_model,
)

In [ ]:
news_collection_name = config["data"]['vector_db']["news"]["news_collection_name"]
news_db_path = os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['vector_db']["location"],config["data"]['vector_db']["news"]["location"])
news_chroma_client = db.PersistentClient(path=news_db_path)
news_langchain_chroma = Chroma(
    client=news_chroma_client,
    collection_name= news_collection_name,
    embedding_function=embedding_model,
)

#### Retrive stocks data to enrich user's portfolio data

In [ ]:
user_stocks = []
for ticker in user_portfolio_df.to_list():
    filter_criteria = {'ticker': {'$eq': ticker}}
    retriever = stocks_langchain_chroma.as_retriever(search_kwargs={"k": 1, "filter": filter_criteria})
    result = retriever.invoke("")[0].metadata
    user_stocks.append(result)

user_stocks_df = pd.DataFrame(user_stocks)

#### Define criteria for information retrieval

In [ ]:
days_threshold = config["rag"]['news']["news_date_threshold"]
top_articles_k = config["rag"]['news']["top_articles_k"]
timestamp_threshold = int((pd.Timestamp.utcnow() - pd.Timedelta(days=days_threshold)).timestamp())
filter_criteria = {'Published': {'$gte': timestamp_threshold}}

#### Retrive most relevant news concerning portfolio

In [ ]:
match_list = []

for i, stock in user_stocks_df.iterrows():
        match_list.append(pd.DataFrame(retrieve_from_news(stock, news_langchain_chroma, mode='stocks', reference_col= 'ticker', filters=filter_criteria, top_k_results=top_articles_k)))
news_x_portfolio_df = pd.concat(match_list, axis=0).merge(user_stocks_df, on='ticker', how='inner')

#### Retrive most relevant news concerning commodities

In [ ]:
match_list = []

for i, commodity in commodities_df.iterrows():
        match_list.append(pd.DataFrame(retrieve_from_news(commodity, news_langchain_chroma, mode='commodities', reference_col= 'name', filters=filter_criteria, top_k_results=top_articles_k)))
news_x_commodities_df = pd.concat(match_list, axis=0).merge(commodities_df, on='name', how='inner')

#### Understand Impact & Perform financial analysis of news on Portfolio

In [ ]:
news_x_portfolio_df['impact'] = news_x_portfolio_df.apply(lambda row: make_news_impact_on_portfolio(row, llm_client), axis=1)

In [ ]:
news_x_portfolio_df['reasons'] = news_x_portfolio_df.apply(lambda row: make_reasons_of_impact_on_portfolio(row, llm_client), axis=1)

#### Understand Impact & Perform financial analysis of news on Commodities

In [ ]:
news_x_commodities_df['impact'] = news_x_commodities_df.apply(lambda row: make_news_impact_on_commodities(row, llm_client), axis=1)

In [ ]:
news_x_commodities_df['reasons'] = news_x_commodities_df.apply(lambda row: make_reasons_of_impact_on_commodities(row, llm_client), axis=1)

#### Process news content and title

In [ ]:
news_x_portfolio_df['news_summary'] = news_x_portfolio_df.apply(lambda row: make_summary_from_news(row, llm_client), axis=1)
news_x_portfolio_df['news_title'] = news_x_portfolio_df.apply(lambda row: make_title_from_news(row, llm_client), axis=1)

In [ ]:
news_x_portfolio_df['news_title'] = news_x_portfolio_df.apply(lambda row: make_title_from_news(row, llm_client), axis=1)

In [ ]:
news_x_commodities_df['news_summary'] = news_x_commodities_df.apply(lambda row: make_summary_from_news(row, llm_client), axis=1)
news_x_commodities_df['news_title'] = news_x_commodities_df.apply(lambda row: make_title_from_news(row, llm_client), axis=1)

#### Output Data for Dashboard

In [ ]:
news_x_portfolio_df.to_csv(os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['dashboard']["location"],config["data"]['dashboard']["filename_portfolio"]), index=False)

In [ ]:
news_x_commodities_df.to_csv(os.path.join(os.path.dirname(os.path.abspath("")), config["data"]["location"], config["data"]['dashboard']["location"],config["data"]['dashboard']["filename_commodities"]), index=False)

In [ ]:
# spark.createDataFrame(news_x_portfolio_df).write.mode("overwrite").saveAsTable("default.dashboard")